<a href="https://colab.research.google.com/github/AmnNrz/llm-extension-training/blob/main/pesticide_efficacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("googledrive")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(googledrive)
library(readr)

In [16]:
# Infection file
inf_id <- "1sHciCtRdE3NvkzL-a3yjShyT_P0W-ysJ"
drive_download(as_id(inf_id), path = "infection.csv", overwrite = TRUE)

# Environmental data (d)
env_d_id <- "1l-TlBzW3SAN4Z_6iBuvXxfxEnEnoIUP0"
drive_download(as_id(env_d_id), path = "environmental.csv", overwrite = TRUE)

# Environmental data (h)
env_h_id <- "1C3uzL0YFilJcCkVmdcR7ETIYfNxrqhsE"
drive_download(as_id(env_h_id), path = "environmental_h.csv", overwrite = TRUE)

File downloaded:

• infections.csv <id: 1sHciCtRdE3NvkzL-a3yjShyT_P0W-ysJ>

Saved locally as:

• infection.csv

File downloaded:

• environmental.csv <id: 1l-TlBzW3SAN4Z_6iBuvXxfxEnEnoIUP0>

Saved locally as:

• environmental.csv

File downloaded:

• environmental_h.csv <id: 1C3uzL0YFilJcCkVmdcR7ETIYfNxrqhsE>

Saved locally as:

• environmental_h.csv



In [17]:
inf_df <- read_csv("infection.csv")
env_d_df <- read_csv("environmental.csv")
env_h_df <- read_csv("environmental_h.csv")

Rows: 566 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): state, variety, location, experiment, trt_code, rep
dbl (7): year, tree_age, clusters, strikes, infection, rel_infection, rel_co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...19`
• `` -> `...20`
Rows: 851 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): state, Date
dbl (16): Year, Day, DayFromInoc, BDD, TRV, Risk, Air_Min_F, Air_Avg_F, Air_...
lgl  (2): ...19, ...20

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 

In [19]:
##############################################
# STEP 1 — Setup & Import (Hourly + Infection)
# - Lightweight: just packages, paths, download, read, and quick checks
##############################################

# --- Packages (keep it simple) ---
# install.packages("googledrive")  # you already installed this
suppressPackageStartupMessages({
  library(googledrive)
  library(readr)    # fast, friendly CSV reader
  library(dplyr)    # for glimpse()
})

# --- File paths (change if you like) ---
INF_PATH  <- "infection.csv"
ENVH_PATH <- "environmental_h.csv"

# --- Google Drive file IDs (yours) ---
inf_id   <- "1sHciCtRdE3NvkzL-a3yjShyT_P0W-ysJ"
env_h_id <- "1C3uzL0YFilJcCkVmdcR7ETIYfNxrqhsE"

# Optional: confirm auth status (will reuse your cached creds if available)
# drive_auth()  # uncomment if you need to re-authenticate

# --- Download the two CSVs (overwrite = TRUE for idempotence) ---
drive_download(as_id(inf_id),  path = INF_PATH,  overwrite = TRUE)
drive_download(as_id(env_h_id), path = ENVH_PATH, overwrite = TRUE)

# --- Read CSVs (no transformations yet) ---
# Note: readr prints a column spec; that's useful for spotting types up front.
inf_df  <- read_csv(INF_PATH)
env_h_df <- read_csv(ENVH_PATH)

# --- Quick sanity checks (no mutation) ---
cat("\n[Infection] rows x cols:", nrow(inf_df), "x", ncol(inf_df), "\n")
cat("[Infection] columns:\n"); print(names(inf_df))

cat("\n[Hourly Env] rows x cols:", nrow(env_h_df), "x", ncol(env_h_df), "\n")
cat("[Hourly Env] columns:\n"); print(names(env_h_df))

cat("\nGlimpse infection:\n"); dplyr::glimpse(inf_df)
cat("\nGlimpse hourly env:\n"); dplyr::glimpse(env_h_df)

# NOTE:
# - No parsing/cleaning yet (that’s Step 2).
# - Keep LeafWetness as-is for now; we’ll drop it in Step 3 per your rules.
##############################################


Auto-refreshing stale OAuth token.

File downloaded:

• infections.csv <id: 1sHciCtRdE3NvkzL-a3yjShyT_P0W-ysJ>

Saved locally as:

• infection.csv

File downloaded:

• environmental_h.csv <id: 1C3uzL0YFilJcCkVmdcR7ETIYfNxrqhsE>

Saved locally as:

• environmental_h.csv

Rows: 566 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): state, variety, location, experiment, trt_code, rep
dbl (7): year, tree_age, clusters, strikes, infection, rel_infection, rel_co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 5447 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (2): state, Date
dbl  (14): Year, Day, DayFromInoc,


[Infection] rows x cols: 566 x 13 
[Infection] columns:
 [1] "year"          "state"         "variety"       "location"     
 [5] "experiment"    "tree_age"      "trt_code"      "rep"          
 [9] "clusters"      "strikes"       "infection"     "rel_infection"
[13] "rel_control"  

[Hourly Env] rows x cols: 5447 x 17 
[Hourly Env] columns:
 [1] "Year"           "state"          "Date"           "Hour"          
 [5] "Day"            "DayFromInoc"    "TRV"            "Risk"          
 [9] "Air_Min_F"      "Air_Avg_F"      "Air_Max_F"      "WindAvg"       
[13] "WindGust"       "Precip"         "SolarRadiation" "Humidity"      
[17] "LeafWetness"   

Glimpse infection:
Rows: 566
Columns: 13
$ year          <dbl> 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 20…
$ state         <chr> "CA", "CA", "CA", "CA", "CA", "CA", "CA", "CA", "CA", "C…
$ variety       <chr> "Shinko", "Shinko", "Shinko", "Shinko", "Shinko", "Shink…
$ location      <chr> "CAShinko", "CAShinko", "CAShinko", "

In [20]:
##############################################
# STEP 2 — Basic parsing & standardization
# - Parse dates/hours (hourly env)
# - Clean types (both tables)
# - Standardize keys and strings
# - Light integrity checks (non-mutating prints)
##############################################

suppressPackageStartupMessages({
  library(dplyr)
  library(stringr)
  library(lubridate)
  library(readr)
})

# ---- Helper: quick NA counter for selected columns ----
na_count <- function(df, cols) {
  df %>%
    summarise(across(all_of(cols), ~sum(is.na(.)), .names = "NA_{.col}"))
}

# ==============================
# 2A) Hourly environmental table
# ==============================
# Expected columns (case-sensitive):
# Year, state, Date, Hour, Day, DayFromInoc, TRV, Risk,
# Air_Min_F, Air_Avg_F, Air_Max_F, WindAvg, WindGust,
# Precip, SolarRadiation, Humidity, LeafWetness

env_h_df <- env_h_df %>%
  # Standardize basic types/casing
  mutate(
    # keys
    Year  = suppressWarnings(as.integer(Year)),
    state = toupper(str_trim(state)),
    # date/time parsing
    Date  = mdy(Date),                                  # "4/26/23" -> Date
    Hour  = suppressWarnings(as.integer(str_extract(Hour, "^[0-9]{1,2}"))),  # "0:00" -> 0
    # anchors & indices
    Day          = suppressWarnings(as.integer(Day)),
    DayFromInoc  = suppressWarnings(as.integer(DayFromInoc))
  ) %>%
  # Coerce numerics (leave LeafWetness for now; drop in Step 3)
  mutate(across(
    c(TRV, Risk, Air_Min_F, Air_Avg_F, Air_Max_F,
      WindAvg, WindGust, Precip, SolarRadiation, Humidity, LeafWetness),
    ~ suppressWarnings(as.numeric(.))
  )) %>%
  # Optional: construct an hourly timestamp (local time, no TZ math)
  mutate(
    h_ts = as_datetime(Date) + hours(coalesce(Hour, 0L)),
    state_year = paste(state, Year, sep = "_")
  )

# Light checks (prints only)
cat("\n[Hourly Env] Unique (Year, state) pairs:", n_distinct(env_h_df$Year, env_h_df$state), "\n")
cat("Sample pairs:\n")
print(env_h_df %>% distinct(Year, state) %>% arrange(Year, state) %>% head(10))

cat("\n[Hourly Env] NA counts (key fields):\n")
print(na_count(env_h_df, c("Year","state","Date","Hour","DayFromInoc","Air_Avg_F","Humidity","Precip")))

# =================
# 2B) Infection table
# =================
# Expected columns:
# year, state, variety, location, experiment, tree_age, trt_code, rep,
# clusters, strikes, infection, rel_infection, rel_control

inf_df <- inf_df %>%
  # Standardize keys, strings
  mutate(
    year  = suppressWarnings(as.integer(year)),
    state = toupper(str_trim(state)),
    variety    = str_squish(as.character(variety)),
    location   = str_squish(as.character(location)),
    experiment = str_squish(as.character(experiment)),
    trt_code   = str_squish(as.character(trt_code)),
    rep        = str_squish(as.character(rep)),
    tree_age   = suppressWarnings(as.integer(tree_age))
  ) %>%
  # Coerce numerics for outcomes and counts
  mutate(across(
    c(clusters, strikes, infection, rel_infection, rel_control),
    ~ suppressWarnings(as.numeric(.))
  )) %>%
  mutate(state_year = paste(state, year, sep = "_"))

# Light checks (prints only)
cat("\n[Infection] Unique (year, state) pairs:", n_distinct(inf_df$year, inf_df$state), "\n")
cat("Sample pairs:\n")
print(inf_df %>% distinct(year, state) %>% arrange(year, state) %>% head(10))

cat("\n[Infection] NA counts (key fields):\n")
print(na_count(inf_df, c("year","state","variety","location","experiment","trt_code","rep",
                         "infection","rel_infection","rel_control")))

# Note:
# - We have *not* filtered to DayFromInoc window yet (that is Step 3).
# - We keep LeafWetness for now (will drop in Step 3).
# - Primary outcome selection will come later (Step 5).
##############################################


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Date = mdy(Date)`.
Caused by warning:
!  1464 failed to parse.”



[Hourly Env] Unique (Year, state) pairs: 7 
Sample pairs:
# A tibble: 7 × 2
   Year state       
  <int> <chr>       
1  2023 MI          
2  2023 NC          
3  2023 NY          
4  2023 WA          
5  2024 EAST LANSING
6  2024 NY          
7  2024 WA          

[Hourly Env] NA counts (key fields):
# A tibble: 1 × 8
  NA_Year NA_state NA_Date NA_Hour NA_DayFromInoc NA_Air_Avg_F NA_Humidity
    <int>    <int>   <int>   <int>          <int>        <int>       <int>
1       0        0    1464      47            455          506           4
# ℹ 1 more variable: NA_Precip <int>

[Infection] Unique (year, state) pairs: 13 
Sample pairs:
# A tibble: 10 × 2
    year state
   <int> <chr>
 1  2022 CA   
 2  2022 MI   
 3  2022 NC   
 4  2022 NY   
 5  2022 WA   
 6  2023 CA   
 7  2023 MI   
 8  2023 NC   
 9  2023 NY   
10  2023 WA   

[Infection] NA counts (key fields):
# A tibble: 1 × 10
  NA_year NA_state NA_variety NA_location NA_experiment NA_trt_code NA_rep
    <int>    <int>      <in

In [28]:
##############################################
# STEP 3 — Window filtering & quality fields
# - Keep DayFromInoc within [-14, +14]
# - Drop LeafWetness
# - Compute coverage/quality per (Year, state) and slice
##############################################

suppressPackageStartupMessages({
  library(dplyr)
  library(tidyr)
  library(stringr)
})

# -----------------------------
# 3A) Filter hourly to analysis window
# -----------------------------
# Keep only rows within [-14, +14], drop LeafWetness, keep everything else
env_h_win <- env_h_df %>%
  filter(!is.na(DayFromInoc),
         DayFromInoc >= -14, DayFromInoc <= 14) %>%
  select(-LeafWetness)

cat("\n[Step 3] Hourly (windowed) rows:", nrow(env_h_win), "\n")

# -----------------------------
# 3B) Coverage / quality by slice
# -----------------------------
# Slices:
#   FULL: DayFromInoc in [-14, +14]
#   PRE : [-7, -1]
#   PERI: [ 0, +3]
#   POST: [+4, +14]
#
# We compute "expected hours" dynamically:
# For each (Year, state, slice), find the number of unique days present (unique DayFromInoc values),
# and multiply by 24 → expected hours. This way, if the dataset only has -7..+10, we don't penalize
# missing days outside that span.
#
# Then coverage:
#   valid_hours = n(), expected_hours = 24 * n_unique_days_present
#   pct_missing = 1 - valid_hours / expected_hours  (bounded to [0,1])

slice_label <- function(dfi) {
  # Helper adds a 'slice' column with labels for each row
  dfi %>%
    mutate(
      slice = case_when(
        DayFromInoc >= -14 & DayFromInoc <= 14 ~ "FULL",
        TRUE ~ NA_character_
      ),
      slice = case_when(
        DayFromInoc >= -7  & DayFromInoc <= -1 ~ "PRE",
        DayFromInoc >= 0   & DayFromInoc <= 3  ~ "PERI",
        DayFromInoc >= 4   & DayFromInoc <= 14 ~ "POST",
        TRUE ~ slice
      )
    )
}

env_h_win_sliced <- env_h_win %>% slice_label()

# Coverage summarizer per (Year, state, slice)
coverage_df <- env_h_win_sliced %>%
  filter(!is.na(slice)) %>%
  group_by(Year, state, slice) %>%
  summarise(
    valid_hours = n_distinct(DayFromInoc, Date, Hour),                                # actual hourly rows present
    n_days_present = n_distinct(DayFromInoc),         # how many distinct DFI days exist in this slice
    expected_hours = 24L * n_days_present,            # expected based on present days
    pct_missing = ifelse(expected_hours > 0,
                         pmax(0, pmin(1, 1 - valid_hours / expected_hours)),
                         NA_real_),
    .groups = "drop"
  )

# Pivot to wide: columns like FULL__valid_hours, PRE__valid_hours, etc.
envh_quality <- coverage_df %>%
  pivot_wider(
    id_cols = c(Year, state),
    names_from = slice,
    values_from = c(valid_hours, expected_hours, pct_missing),
    names_glue = "{slice}__{.value}"
  ) %>%
  arrange(Year, state)

# Optional: fill missing slice columns with zeros/NA for cleaner downstream use
# (Here we keep as-is to avoid hiding gaps; you can coalesce later if needed.)

# Quick peek
cat("\n[Step 3] Coverage summary (first 10 rows):\n")
print(envh_quality %>% head(10))

# Notes:
# - We used only the days that actually exist in each slice to define "expected hours".
# - This prevents penalizing days outside your actual collection span (e.g., -7..+10).
# - Keep env_h_win for feature construction in Step 4.
##############################################



[Step 3] Hourly (windowed) rows: 3888 

[Step 3] Coverage summary (first 10 rows):
# A tibble: 7 × 14
   Year state        PERI__valid_hours POST__valid_hours PRE__valid_hours
  <int> <chr>                    <int>             <int>            <int>
1  2023 MI                          96               240              168
2  2023 NC                          96               264              168
3  2023 NY                          96               264              168
4  2023 WA                          96               263              168
5  2024 EAST LANSING                96               264              216
6  2024 NY                          96               264              168
7  2024 WA                          52               143               NA
# ℹ 9 more variables: FULL__valid_hours <int>, PERI__expected_hours <int>,
#   POST__expected_hours <int>, PRE__expected_hours <int>,
#   FULL__expected_hours <int>, PERI__pct_missing <dbl>,
#   POST__pct_missing <dbl>, PRE__pct_mi

In [29]:
envh_quality

Year,state,PERI__valid_hours,POST__valid_hours,PRE__valid_hours,FULL__valid_hours,PERI__expected_hours,POST__expected_hours,PRE__expected_hours,FULL__expected_hours,PERI__pct_missing,POST__pct_missing,PRE__pct_missing,FULL__pct_missing
<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2023,MI,96,240,168,NA,96,240,168,NA,0.0000000,0.000000000,0,NA
2023,NC,96,264,168,NA,96,264,168,NA,0.0000000,0.000000000,0,NA
2023,NY,96,264,168,NA,96,264,168,NA,0.0000000,0.000000000,0,NA
2023,WA,96,263,168,NA,96,264,168,NA,0.0000000,0.003787879,0,NA
2024,EAST LANSING,96,264,216,168,96,264,168,168,0.0000000,0.000000000,0,0
2024,NY,96,264,168,168,96,264,168,168,0.0000000,0.000000000,0,0
2024,WA,52,143,NA,NA,96,264,NA,NA,0.4583333,0.458333333,NA,NA


In [30]:
##############################################
# STEP 4 (fixed) — Starter-pack hourly features (per state–year)
# - Slices computed independently:
#   FULL: DayFromInoc ∈ [-14, +14]
#   PRE : [-7, -1]
#   PERI: [ 0, +3]
# - Distinct (DayFromInoc, Date, Hour) used for hour counts
# - Risk band: 68–75 °F on Air_Avg_F
# - Robust guards: NaN/Inf -> NA
# - Local state cleanup: "EAST LANSING" -> "MI"
##############################################

suppressPackageStartupMessages({
  library(dplyr)
  library(tidyr)
  library(purrr)
})

# --- Small helpers for robust stats ---
safe_mean <- function(x) {
  out <- suppressWarnings(mean(x, na.rm = TRUE))
  if (is.nan(out)) NA_real_ else out
}
safe_sd <- function(x) {
  out <- suppressWarnings(sd(x, na.rm = TRUE))
  if (is.nan(out)) NA_real_ else out
}
safe_max <- function(x) {
  out <- suppressWarnings(max(x, na.rm = TRUE))
  if (!is.finite(out)) NA_real_ else out
}

# --- Local, non-destructive state cleanup for obvious anomalies ---
env_h_fix <- env_h_win %>%
  mutate(state = case_when(
    state == "EAST LANSING" ~ "MI",   # fix anomaly seen in your printout
    TRUE ~ state
  ))

# --- Define independent slice filters ---
full_df <- env_h_fix %>% filter(DayFromInoc >= -14, DayFromInoc <= 14)
pre_df  <- env_h_fix %>% filter(DayFromInoc >=  -7, DayFromInoc <=  -1)
peri_df <- env_h_fix %>% filter(DayFromInoc >=   0, DayFromInoc <=   3)

# --- FULL features ---
# Use distinct (DayFromInoc, Date, Hour) per site-year for valid hours calc
full_feat <- full_df %>%
  mutate(is_risk_band = !is.na(Air_Avg_F) & Air_Avg_F >= 68 & Air_Avg_F <= 75) %>%
  group_by(Year, state) %>%
  summarise(
    FULL__valid_hours_calc = n_distinct(DayFromInoc, Date, Hour),
    FULL__Air_Avg_F__mean  = safe_mean(Air_Avg_F),
    FULL__Air_Avg_F__std   = safe_sd(Air_Avg_F),
    FULL__hours_temp_68_75__count = sum(is_risk_band, na.rm = TRUE),
    FULL__hours_temp_68_75__frac  = ifelse(FULL__valid_hours_calc > 0,
                                           FULL__hours_temp_68_75__count / FULL__valid_hours_calc, NA_real_),
    FULL__Precip__sum         = sum(Precip, na.rm = TRUE),
    FULL__WindGust__max       = safe_max(WindGust),
    FULL__SolarRadiation__sum = sum(SolarRadiation, na.rm = TRUE),
    .groups = "drop"
  )

# --- PERI features ---
peri_feat <- peri_df %>%
  group_by(Year, state) %>%
  summarise(
    PERI__Air_Avg_F__mean = safe_mean(Air_Avg_F),
    PERI__TRV__sum        = sum(TRV,  na.rm = TRUE),
    PERI__Risk__sum       = sum(Risk, na.rm = TRUE),
    .groups = "drop"
  )

# --- PRE features ---
pre_feat <- pre_df %>%
  group_by(Year, state) %>%
  summarise(
    PRE__Humidity__mean = safe_mean(Humidity),
    .groups = "drop"
  )

# --- Diurnal swing over FULL days ---
# Average of (daily max Air_Max_F − daily min Air_Min_F) for each DayFromInoc
diurnal_full <- full_df %>%
  group_by(Year, state, DayFromInoc) %>%
  summarise(
    daily_max = safe_max(Air_Max_F),
    daily_min = suppressWarnings(min(Air_Min_F, na.rm = TRUE)),
    .groups = "drop"
  ) %>%
  mutate(
    daily_min = ifelse(is.infinite(daily_min), NA_real_, daily_min),
    daily_range = ifelse(is.na(daily_max) | is.na(daily_min), NA_real_, daily_max - daily_min)
  ) %>%
  group_by(Year, state) %>%
  summarise(
    FULL__diurnal_range__mean_by_day = safe_mean(daily_range),
    .groups = "drop"
  )

# --- Bring coverage/quality (from Step 3) ---
quality_cols <- envh_quality %>%
  # also fix state anomaly here to ensure join aligns
  mutate(state = case_when(
    state == "EAST LANSING" ~ "MI",
    TRUE ~ state
  ))

# --- Assemble final hourly feature table ---
env_hourly_features <- full_feat %>%
  full_join(peri_feat,    by = c("Year", "state")) %>%
  full_join(pre_feat,     by = c("Year", "state")) %>%
  full_join(diurnal_full, by = c("Year", "state")) %>%
  full_join(quality_cols, by = c("Year", "state")) %>%
  arrange(Year, state)

# Quick peek
cat("\n[Step 4 FIXED] Hourly feature table — rows x cols:",
    nrow(env_hourly_features), "x", ncol(env_hourly_features), "\n")
print(env_hourly_features %>% select(Year, state, FULL__valid_hours_calc,
                                     FULL__Air_Avg_F__mean,
                                     FULL__hours_temp_68_75__count,
                                     FULL__hours_temp_68_75__frac) %>% head(10))

# Notes:
# - Slices computed independently; FULL now truly covers [-14..+14] hours present.
# - Hour counts use distinct (DayFromInoc, Date, Hour) to avoid duplicates/DST effects.
# - NaN/Inf guarded to return NA instead of -Inf/NaN.
# - A minimal state cleanup maps "EAST LANSING" -> "MI" for joining.
##############################################



[Step 4 FIXED] Hourly feature table — rows x cols: 7 x 27 
# A tibble: 7 × 6
   Year state FULL__valid_hours_calc FULL__Air_Avg_F__mean
  <int> <chr>                  <int>                 <dbl>
1  2023 MI                       504                  NA  
2  2023 NC                       528                  55.9
3  2023 NY                       528                  56.6
4  2023 WA                       527                  65.1
5  2024 MI                       744                  57.1
6  2024 NY                       696                  58.3
7  2024 WA                       195                  52.6
# ℹ 2 more variables: FULL__hours_temp_68_75__count <int>,
#   FULL__hours_temp_68_75__frac <dbl>


In [35]:
colnames(env_hourly_features)

[1] "Year"                             "state"                           
 [3] "FULL__valid_hours_calc"           "FULL__Air_Avg_F__mean"           
 [5] "FULL__Air_Avg_F__std"             "FULL__hours_temp_68_75__count"   
 [7] "FULL__hours_temp_68_75__frac"     "FULL__Precip__sum"               
 [9] "FULL__WindGust__max"              "FULL__SolarRadiation__sum"       
[11] "PERI__Air_Avg_F__mean"            "PERI__TRV__sum"                  
[13] "PERI__Risk__sum"                  "PRE__Humidity__mean"             
[15] "FULL__diurnal_range__mean_by_day" "PERI__valid_hours"               
[17] "POST__valid_hours"                "PRE__valid_hours"                
[19] "FULL__valid_hours"                "PERI__expected_hours"            
[21] "POST__expected_hours"             "PRE__expected_hours"             
[23] "FULL__expected_hours"             "PERI__pct_missing"               
[25] "POST__pct_missing"                "PRE__pct_missing"                
[27] "FULL__pct_missing"

In [41]:
##############################################
# STEP 5 — Prepare infection table
# - Outcome (Y): infection
# - Cohort: variety == "gala"
# - Keep essential keys + tidy outcome name
##############################################

suppressPackageStartupMessages({
  library(dplyr)
  library(stringr)
})

# --- 5A) Filter to 'gala' and select essential columns ---
# Note: we treat variety matching case-insensitively, then standardize to "gala"
inf_gala <- inf_df %>%
  mutate(variety = str_squish(tolower(as.character(variety)))) %>%
  filter(variety == "gala") %>%
  # Keep essential keys and the outcome columns we need
  select(
    year, state, variety, location, experiment, tree_age,
    trt_code, rep,
    infection, rel_infection, rel_control
  ) %>%
  # Tidy outcome name for clarity; keep originals too (can drop later)
  mutate(y_infection = infection)

# --- 5B) Light consistency checks (prints only) ---
cat("\n[Step 5] Infection (gala) — rows x cols:", nrow(inf_gala), "x", ncol(inf_gala), "\n")

cat("\n[Step 5] NA counts (key fields):\n")
print(
  inf_gala %>%
    summarise(
      NA_year        = sum(is.na(year)),
      NA_state       = sum(is.na(state)),
      NA_location    = sum(is.na(location)),
      NA_experiment  = sum(is.na(experiment)),
      NA_trt_code    = sum(is.na(trt_code)),
      NA_rep         = sum(is.na(rep)),
      NA_tree_age    = sum(is.na(tree_age)),
      NA_infection   = sum(is.na(infection))
    )
)

cat("\n[Step 5] Sample rows (head):\n")
print(inf_gala %>% head(8))

# NOTE:
# - We kept rel_infection and rel_control for reference only.
#   The primary Y for analysis is y_infection.
# - If you want to tag the inoculated check (water-only) treatment later,
#   we can add an `is_check` column once you provide the code(s).
##############################################



[Step 5] Infection (gala) — rows x cols: 408 x 12 

[Step 5] NA counts (key fields):
# A tibble: 1 × 8
  NA_year NA_state NA_location NA_experiment NA_trt_code NA_rep NA_tree_age
    <int>    <int>       <int>         <int>       <int>  <int>       <int>
1       0        0           0             0           0      0           0
# ℹ 1 more variable: NA_infection <int>

[Step 5] Sample rows (head):
# A tibble: 8 × 12
   year state variety location experiment tree_age trt_code      rep   infection
  <int> <chr> <chr>   <chr>    <chr>         <int> <chr>         <chr>     <dbl>
1  2022 MI    gala    MIGala   MIGala2022        8 Alum          C             4
2  2022 MI    gala    MIGala   MIGala2022        8 Alum          D             5
3  2022 MI    gala    MIGala   MIGala2022        8 Alum          A             6
4  2022 MI    gala    MIGala   MIGala2022        8 Alum          B             9
5  2022 MI    gala    MIGala   MIGala2022        8 BacterioPhage A            38
6  2022 MI  

In [43]:
##############################################
# STEP 6 — Join infection (gala) with hourly features
# - Keys: (year == Year, state)
# - Keep all infection rows (left join)
# - Sanity checks + save outputs
##############################################

suppressPackageStartupMessages({
  library(dplyr)
  library(readr)
})

# --- 6A) Confirm key types/casing (already standardized earlier) ---
# Infection side: year (int), state (UPPER), variety == "gala"
# Weather features side: Year (int), state (UPPER)
cat("\n[Step 6] Key preview:\n")
print(inf_gala %>% distinct(year, state) %>% arrange(year, state) %>% head(10))
print(env_hourly_features %>% distinct(Year, state) %>% arrange(Year, state) %>% head(10))

# --- 6B) Left-join infection -> features ---
inf_gala_hourly_joined <- inf_gala %>%
  left_join(env_hourly_features, by = c("year" = "Year", "state" = "state"))

# --- 6C) Sanity checks after join ---
cat("\n[Step 6] Joined table — rows x cols:",
    nrow(inf_gala_hourly_joined), "x", ncol(inf_gala_hourly_joined), "\n")

# Any rows without features?
no_match <- inf_gala_hourly_joined %>%
  filter(is.na(FULL__Air_Avg_F__mean) &
         is.na(PERI__TRV__sum) &
         is.na(FULL__hours_temp_68_75__frac))

n_nm <- nrow(no_match)
cat("[Step 6] Infection rows without matching features:", n_nm, "\n")

if (n_nm > 0) {
  cat("[Step 6] Unmatched (year, state) combos (first 10):\n")
  print(no_match %>% distinct(year, state) %>% head(10))
}

# Spot-check a few feature columns
cat("\n[Step 6] Feature column spot check (first 8 rows):\n")
print(
  inf_gala_hourly_joined %>%
    select(year, state, location, experiment, trt_code, rep, y_infection,
           FULL__Air_Avg_F__mean, PERI__TRV__sum, FULL__hours_temp_68_75__frac,
           FULL__valid_hours, FULL__pct_missing) %>%
    head(8)
)

# --- 6D) Save outputs ---
# 1) Features-only table (state-year)
write_csv(env_hourly_features, "hourly_features_state_year.csv")

# 2) Final analysis table (infection + hourly features)
write_csv(inf_gala_hourly_joined, "infection_gala_with_hourly_features.csv")

cat("\n[Step 6] Saved:\n",
    "- hourly_features_state_year.csv\n",
    "- infection_gala_with_hourly_features.csv\n")
##############################################



[Step 6] Key preview:
# A tibble: 10 × 2
    year state
   <int> <chr>
 1  2022 MI   
 2  2022 NC   
 3  2022 NY   
 4  2023 MI   
 5  2023 NC   
 6  2023 NY   
 7  2023 WA   
 8  2024 MI   
 9  2024 NY   
10  2024 WA   
# A tibble: 7 × 2
   Year state
  <int> <chr>
1  2023 MI   
2  2023 NC   
3  2023 NY   
4  2023 WA   
5  2024 MI   
6  2024 NY   
7  2024 WA   

[Step 6] Joined table — rows x cols: 408 x 37 
[Step 6] Infection rows without matching features: 104 
[Step 6] Unmatched (year, state) combos (first 10):
# A tibble: 3 × 2
   year state
  <int> <chr>
1  2022 MI   
2  2022 NC   
3  2022 NY   

[Step 6] Feature column spot check (first 8 rows):
# A tibble: 8 × 12
   year state location experiment trt_code      rep   y_infection
  <int> <chr> <chr>    <chr>      <chr>         <chr>       <dbl>
1  2022 MI    MIGala   MIGala2022 Alum          C               4
2  2022 MI    MIGala   MIGala2022 Alum          D               5
3  2022 MI    MIGala   MIGala2022 Alum          A      

In [46]:

colnames(inf_gala_hourly_joined)

[1] "year"                             "state"                           
 [3] "variety"                          "location"                        
 [5] "experiment"                       "tree_age"                        
 [7] "trt_code"                         "rep"                             
 [9] "infection"                        "rel_infection"                   
[11] "rel_control"                      "y_infection"                     
[13] "FULL__valid_hours_calc"           "FULL__Air_Avg_F__mean"           
[15] "FULL__Air_Avg_F__std"             "FULL__hours_temp_68_75__count"   
[17] "FULL__hours_temp_68_75__frac"     "FULL__Precip__sum"               
[19] "FULL__WindGust__max"              "FULL__SolarRadiation__sum"       
[21] "PERI__Air_Avg_F__mean"            "PERI__TRV__sum"                  
[23] "PERI__Risk__sum"                  "PRE__Humidity__mean"             
[25] "FULL__diurnal_range__mean_by_day" "PERI__valid_hours"               
[27] "POST__valid_hours"                "PRE__valid_hours"                
[29] "FULL__valid_hours"                "PERI__expected_hours"            
[31] "POST__expected_hours"             "PRE__expected_hours"             
[33] "FULL__expected_hours"             "PERI__pct_missing"               
[35] "POST__pct_missing"                "PRE__pct_missing"                
[37] "FULL__pct_missing"

In [49]:
inf_gala_hourly_joined$FULL__WindGust__max

[1]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [13]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [25]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [37]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [49]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [61]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [73]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [85]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
 [97]    NA    NA    NA    NA    NA    NA    NA    NA 25.60 25.60 25.60 25.60
[109] 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60
[121] 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60
[133] 25.60 25.60 25.60 25.60 25.60 25.60 25.60 25.60 21.30 21.30 21.30 21.30
[145] 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30
[157] 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30
[169] 21.30 21.30 21.30 21.30 21.30 21.30 21.30 21.30    NA    NA    NA    NA
[181]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
[193]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
[205]    NA    NA    NA    NA    NA    NA    NA    NA 31.40 31.40 31.40 31.40
[217] 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40
[229] 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40
[241] 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40
[253] 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40 31.40
[265] 31.40 31.40 31.40    NA    NA    NA    NA    NA    NA    NA    NA    NA
[277]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
[289]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
[301]    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA
[313]    NA    NA    NA    NA    NA 33.44 33.44 33.44 33.44 33.44 33.44 33.44
[325] 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44
[337] 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44 33.44
[349] 33.44 33.44 33.44 33.44 33.44 21.80 21.80 21.80 21.80 21.80 21.80 21.80
[361] 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80
[373] 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80
[385] 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80
[397] 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80 21.80